In [ ]:
import shutil
import glob

In [ ]:
train_dr = '/content/Croneal Ulcer/train' 
org_dir =  '/content/Croneal Ulcer/train' 

In [ ]:
train_dr

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import  Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#create images with backgrounds. 
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive =  GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q': "title = 'Eye Disease Deep Learning Dataset (2).zip' "}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Eye Disease Deep Learning Dataset (2).zip')

In [ ]:
!unzip 'Eye Disease Deep Learning Dataset (2).zip'

In [ ]:
#path = '/content/Croneal Ulcer'
path = '/content/Eye Disease Deep Learning Dataset'

In [ ]:
os.listdir(path)

In [ ]:
#train_dir = os.path.join(path, '/content/Croneal Ulcer/train ')
train_dir = os.path.join(path, 'train')

In [ ]:
test_dir = os.path.join(path, 'test')

In [ ]:
train_cat_0 = os.path.join(train_dir, '0')
train_cat_1 = os.path.join(train_dir, '1')
train_cat_2 = os.path.join(train_dir, '2')

test_cat_0 = os.path.join(test_dir, '0')
test_cat_1 = os.path.join(test_dir, '1')
test_cat_2 = os.path.join(test_dir, '2')

In [ ]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))


print(len(os.listdir(train_cat_0)))
print(len(os.listdir(train_cat_1)))
print(len(os.listdir(train_cat_2)))

print(len(os.listdir(test_cat_0)))
print(len(os.listdir(test_cat_1)))
print(len(os.listdir(test_cat_2)))

In [ ]:
img_height = 1728
img_width = 2592
batch_size = 32

In [ ]:
img_gen = ImageDataGenerator(
    rescale = 1/255
)
img_gen

this function will plot images inthe form of a grid with 1 row and 5 coulms where images are placed in each column,

In [ ]:
train_data_gen = img_gen.flow_from_directory(
    directory = train_dir,
    shuffle = True,
    #traget_size = ('img_height','img_width'),
    target_size =(1728, 2592),
    class_mode = 'categorical' #  sparse', 'categorical', 'input', None, 'binary
)

In [ ]:
test_data_gen = img_gen.flow_from_directory(
    directory = test_dir,
    shuffle = True,
    #traget_size = (img_height, img_width),
    target_size =(1728, 2592),
    class_mode = 'categorical' #  sparse', 'categorical', 'input', None, 'binary
)

In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
def  plotImages(images_arr):
  fig, axes = plt.subplots(1,3, figsize = (20,20))
  axes = axes.flatten()
  for img, ax in zip( images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages(sample_training_images[:3])

In [ ]:
sample_testing_images, _ = next(test_data_gen)

In [ ]:
plotImages(sample_testing_images[:3])

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding = 'same', activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 32 #there wiil be 32 images per batch / per cycle of trainning 512/32 = 16 iteration in trainning per batch.
epochs =20

In [ ]:
num_0_train = len(os.listdir(train_cat_0))
num_1_train = len(os.listdir(train_cat_1))
num_2_train = len(os.listdir(train_cat_2))

num_0_test = len(os.listdir(test_cat_0))
num_1_test = len(os.listdir(test_cat_1))
num_2_test = len(os.listdir(test_cat_2))

total_train = num_0_train + num_1_train + num_2_train
total_test = num_0_test + num_1_test + num_2_test

print(total_train)
print(total_test)

In [ ]:

#  the model , intended for validation
history = model.fit(
    train_data_gen,
    steps_per_epoch = total_train // 16,
    epochs = 20,
    validation_data = test_data_gen,
    validation_steps = total_test // 16,
    callbacks = [ tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0.01,
        patience =7)]
)


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch, 
                         y=history.history['accuracy'],
                         mode ='lines + markers',
                         name='Training accuracy'
                         ))
fig.add_trace(go.Scatter(x = history.epoch,
                         y = history.history['val_accuracy'],
                         mode ='lines + markers',
                         name ='Validation accuracy'
                         ))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))
fig.show()

In [ ]:
save_path = F'/content/drive/MyDrive/Dataset/Classification'

In [ ]:
model.save_weights(os.path.join(save_path, 'Eye Disease Deep Learning Dataset.h5)'))

In [ ]:
model_json = model.to_json()

In [ ]:
with open(os.path.join(save_path, 'Eye Disease Deep Learning Dataset.json'), 'w') as json_file:
  json_file.write(model_json)

In [ ]:
json_file.close()

In [ ]:
from skimage.transform import resize #resize image 

In [ ]:
my_image = plt.imread("/content/Eye Disease Deep Learning Dataset/train/0/1.jpg")

#resize the image to fit model 
#model image size = 32*32*3
my_image_resized = resize(my_image, (1728, 2592,3)) # already makes values between 0-1
#visualize the image 
img = plt.imshow(my_image) #will only show one 


In [ ]:
probabilities = model.predict(np.array([my_image_resized]))

In [ ]:
number=['0',\
        '1',\
        '2',]

In [ ]:
#prediction analysing 
index = np.argsort(probabilities[0,:])
print("1st Most likely class:", number[index[2]], "-- Probability:", probabilities[0, index[2] ] )
print("2nd Most likely class:", number[index[1]], "-- Probability:", probabilities[0, index[1] ] )
print("3rd Most likely class:", number[index[0]], "-- Probability:", probabilities[0, index[0] ] )